## קוד מבוא

In [1]:
import os
import pathlib
import pandas as pd
import geopandas as gpd
import fiona

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

## פונקציות

In [3]:
def double_taz_num(df):
    dup_taz_num=df.groupby(['Taz_num']).size().reset_index(name='count').query('count>1').Taz_num.to_list()
    return df.loc[df['Taz_num'].isin(dup_taz_num)]

In [4]:
def make_point(df):
    df_point=df.copy()
    df_point['centroid'] = df_point.representative_point()
    df_point=df_point.set_geometry('centroid')
    df_point=df_point.drop(columns=['geometry'],axis=1)
    return df_point

In [5]:
def up_load_gdb(path,layer_name):
    path='{}'.format(path)
    layer_list=fiona.listlayers(path)
    gpd_layer=gpd.read_file(path, layer=layer_list.index(layer_name))
    return gpd_layer

In [6]:
def up_load_shp(path):
    path='{}'.format(path)
    gpd_layer=gpd.read_file(path)
    return gpd_layer

In [7]:
def unique_id_in_one_taz(df_to_geoode,unique_field,gpd_for_geocode,taz):
    
    code_to_find=list(df_to_geoode[unique_field].unique())

    gpd_for_geocode_to_sum_by_taz=gpd_for_geocode[[unique_field,'geometry']].loc[gpd_for_geocode[unique_field].isin(code_to_find)]

    gpd_for_geocode_to_sum_by_taz=gpd.sjoin(taz[['Taz_num','geometry']],gpd_for_geocode_to_sum_by_taz)

    gpd_for_geocode_in_one_taz=list(gpd_for_geocode_to_sum_by_taz.groupby(unique_field)[['Taz_num']].nunique().query('Taz_num==1').reset_index()[unique_field])

    return gpd_for_geocode_in_one_taz

In [8]:
def up_load_df(folder_path,file_name):
    
    path_df=r'{}\{}.xlsx'.format(folder_path,file_name)
    df=pd.read_excel(path_df)
    df=df.dropna(how='all')

    return df

In [9]:
def find_files_with_pattern(folder_path, pattern):
    """
    Find files in a directory that match a certain pattern.
    
    Args:
    - directory (str): The directory path.
    - pattern (str): The pattern to search for in file names.
    
    Returns:
    - List of file paths matching the pattern.
    """
    files = []
    for root, _, filenames in os.walk(folder_path):
        for filename in filenames:
            if pattern in filename:
                files.append(os.path.join(root, filename))
    return files

## code

### עיבוד חומר מקדים

In [10]:
#העלת משתנים להרצת הקוד
df_inputs_outputs = pd.read_excel('inputs_outputs.xlsx')

software_data_folder_location=df_inputs_outputs['location'][0]

forecast_version_folder_location=df_inputs_outputs['location'][1]

In [11]:
path=r'{}\background_files'.format(software_data_folder_location)

In [12]:
v_date='230720'

In [13]:
taz_2020_catalog=up_load_df(path,'forecast_2020_{}_age_dis_type'.format(v_date))

matching_files=find_files_with_pattern(r'{}'.format(forecast_version_folder_location), '2020_jtmt_forcast_full')

file = pathlib.Path(matching_files[0])
file_name = file.name

taz_2020_aprt=up_load_df(forecast_version_folder_location, file_name[:-5])

In [14]:
save_file=r'{}\Intermediates'.format(software_data_folder_location)

matching_files=find_files_with_pattern(r'{}'.format(save_file), 'kibolt_jew_till_2050')

file = pathlib.Path(matching_files[0])
file_name = file.name
folder = str(file.parent)
file_name[:-5]

kibolt_jew_2050=up_load_df(save_file, file_name[:-5]).set_index('Taz_num')

In [15]:
age_catalog=up_load_df(r'{}\background_files'.format(software_data_folder_location),'age_des_types')

taz_with_aprt_diversity=up_load_shp(r'{}\background_files\cost_diversity_for_age_distribution.shp'.format(software_data_folder_location)).set_index('Taz_num')

taz_2020_catalog=taz_2020_catalog.merge(taz_2020_aprt[['Taz_num','zonetype','aprt_20','pop_without_dorms_yeshiva','Taz_name','Muni_Heb']],on='Taz_num',how='left').set_index('Taz_num')

taz_2020_catalog['taz_with_aprt_diversity']=taz_with_aprt_diversity['div']

taz_2020_catalog['taz_expensive']=taz_with_aprt_diversity['cost']

col=['classification_name','age_cycle']

taz_2020_catalog=taz_2020_catalog.reset_index().merge(age_catalog[col],on='classification_name',how='left').set_index('Taz_num')

taz_2020_catalog=taz_2020_catalog.join(kibolt_jew_2050).query('sector_for_age != "Arab"')

taz_2020_catalog=taz_2020_catalog.fillna(0)

## חישוב 

In [16]:
singal_year=list(range(2025,2051,5))

In [17]:
years=['add_aprt_2020_2025',
 'add_aprt_2025_2030',
 'add_aprt_2030_2035',
 'add_aprt_2035_2040',
 'add_aprt_2040_2045',
 'add_aprt_2045_2050']

y_before=['2020','2020_2025',
 '2025_2030',
 '2030_2035',
 '2035_2040',
 '2040_2045']

only_years=['2020_2025',
 '2025_2030',
 '2030_2035',
 '2035_2040',
 '2040_2045',
 '2045_2050']

In [18]:
taz_2020_catalog['age_cycle_time']=0
taz_2020_catalog['current_aprt']=taz_2020_catalog['aprt_20']
years_to_change_cycle_stage=20
age_cycle_limit=4
taz_2020_catalog['age_cycle_2020']=taz_2020_catalog['age_cycle']
taz_2020_catalog['changed_this_cycle']=0

In [19]:
for y, y1 in zip(years, only_years):
    
        
    
    #### אזורים ללא יח"ד קיימות בשנת הבסיס מתאכלסים בהתפלגות גילים הראשוני

    taz_with_aprt_diversity_condition = taz_2020_catalog['taz_with_aprt_diversity']==1
    taz_expensive_condition = taz_2020_catalog['taz_expensive']==1
    

    taz_2020_catalog.loc[
        (taz_2020_catalog['current_aprt']==0)&
        (taz_2020_catalog['{}'.format(y)]>0)&
        (taz_2020_catalog['changed_this_cycle']==0)&
        taz_with_aprt_diversity_condition,['age_cycle','age_cycle_time','explanation_{}'.format(y1),'changed_this_cycle']
    ]=[3,0,' | new diverse neighborhood | ',1]

    
    taz_2020_catalog.loc[
        (taz_2020_catalog['current_aprt']==0)&
        (taz_2020_catalog['{}'.format(y)]>0)&
        (taz_2020_catalog['changed_this_cycle']==0)&
        taz_expensive_condition&
        ~taz_with_aprt_diversity_condition,['age_cycle','age_cycle_time','explanation_{}'.format(y1),'changed_this_cycle']
    ]=[2,0,' | new non-diverse expensive neighborhood  | ',1]
    
    taz_2020_catalog.loc[
        (taz_2020_catalog['current_aprt']==0)&
        (taz_2020_catalog['{}'.format(y)]>0)&
        (taz_2020_catalog['changed_this_cycle']==0)&
        ~taz_expensive_condition&
        ~taz_with_aprt_diversity_condition,['age_cycle','age_cycle_time','explanation_{}'.format(y1),'changed_this_cycle']
    ]=[1,0,' | new non-diverse neighborhood  | ',1]

    #### אזורים עם יח"ד קיימות בשנת הבסיס מתאכלסים בהתפלגות גילים כפונצקיה של יחס קיבולת למצב הקיים

    taz_2020_catalog.loc[
        (taz_2020_catalog['current_aprt']>0),'precent_add_to_current'
    ]=taz_2020_catalog['{}'.format(y)]/taz_2020_catalog['current_aprt']


    
    
    
    #### יחס נמוך - הקיבולת בטלה ביחס למצב הקיים
        #### יחס גדול - הקיבולת מבטלת את המצב הקיים

    kibolet_condition = taz_2020_catalog['precent_add_to_current'] > 0.5
    yush_condition =  taz_2020_catalog['zonetype']=='Judea and Samaria'

    taz_2020_catalog.loc[
        kibolet_condition &
        ~yush_condition &
        (taz_2020_catalog['changed_this_cycle']==0)&
        taz_with_aprt_diversity_condition,['age_cycle','age_cycle_time','explanation_{}'.format(y1),'changed_this_cycle']
    ]=[3,0,'| renew diverse neighborhood |',1]
    
    taz_2020_catalog.loc[
        kibolet_condition &
        taz_expensive_condition &
        (taz_2020_catalog['changed_this_cycle']==0)&
        ~taz_with_aprt_diversity_condition,['age_cycle','age_cycle_time','explanation_{}'.format(y1),'changed_this_cycle']
    ]=[2,0,'| a lot of new aprts in non-diverse expensive neighborhood|',1]

    taz_2020_catalog.loc[
        kibolet_condition &
        (taz_2020_catalog['changed_this_cycle']==0)&
        ~taz_expensive_condition &
        ~taz_with_aprt_diversity_condition,['age_cycle','age_cycle_time','explanation_{}'.format(y1),'changed_this_cycle']
    ]=[1,0,'| a lot of new aprts in non-diverse neighborhood|',1]

    #### יחס בינוני

    ##### ביוש אין התחדשות עירונית לכן אם יחס גבוהה יחסית זה כמו בנייה חדשה עם אכלוס של סוג התפלגות גילים הראשוני

    
    kibolet_condition =  taz_2020_catalog['precent_add_to_current']>= 0.20

    taz_2020_catalog.loc[
        (yush_condition)&
        (taz_2020_catalog['changed_this_cycle']==0)&
        (kibolet_condition) , ['age_cycle','age_cycle_time','explanation_{}'.format(y1),'changed_this_cycle']
    ] = [1, 0,'| a lot of new aprts in non-diverse neighborhood in yush |',1]


    ##### יוש אין התחדשות עירונית לכן אם יחס לא גבוהה יחסית זה מונע התבגרות של המצב הקיים
    

    ##### אם לא ביוש אז יש אפשרות התחדשות עירונית לכן המקום נהיה מגוון

    
    kibolet_condition =  (taz_2020_catalog['precent_add_to_current']>= 0.25) & (taz_2020_catalog['precent_add_to_current'] <= 0.5)
    

    taz_2020_catalog.loc[
        ~(yush_condition)&
        (kibolet_condition)&
        (taz_with_aprt_diversity_condition) , ['age_cycle','explanation_{}'.format(y1)]
    ] =  [3,'| a lot of new aprts in diverse neighborhood|']
    
    
    
    #השעון התקדם כבר ולא צריך לשנות כלום

    #### קוד לתהליך התבגרות התפלגות הגילים כפונקציה של סוג מגזר

    

    age_cycle_condition = taz_2020_catalog['age_cycle']<age_cycle_limit

    age_cycle_time_condition=(taz_2020_catalog['age_cycle_time']%years_to_change_cycle_stage==0)&(taz_2020_catalog['age_cycle_time']>0)

    sector_condition=taz_2020_catalog['sector_for_age']=='hitnachlut' 
    
    taz_2020_catalog.loc[
        (age_cycle_time_condition) &
        (sector_condition) &
        (taz_2020_catalog['changed_this_cycle']==0)&
        (age_cycle_condition),['explanation_{}'.format(y1),'changed_this_cycle']
    ]=['| natural aging |',1]
    
    taz_2020_catalog.loc[
        (age_cycle_time_condition) &
        (sector_condition) &
        (age_cycle_condition),'age_cycle'
    ]=taz_2020_catalog['age_cycle']+1
    
    
    taz_2020_catalog.loc[
        (age_cycle_time_condition) &
        (sector_condition) &
        (age_cycle_condition),'age_cycle_time'
    ]=taz_2020_catalog['age_cycle_time']+5

    sector_condition=taz_2020_catalog['sector_for_age']=='Jewish'

    age_cycle_condition = taz_2020_catalog['age_cycle']==1


    taz_2020_catalog.loc[
        (age_cycle_time_condition) &
        (taz_2020_catalog['changed_this_cycle']==0)&
        (sector_condition) &
        (age_cycle_condition),['age_cycle','explanation_{}'.format(y1),'changed_this_cycle']
    ]=[2 ,'| natural aging |',1]
    
    
    taz_2020_catalog.loc[
        (age_cycle_time_condition) & 
        (sector_condition) &
        (age_cycle_condition),'age_cycle_time'
    ]=taz_2020_catalog['age_cycle_time']+5


    sector_condition=taz_2020_catalog['sector_for_age']=='Jewish'

    age_cycle_condition = taz_2020_catalog['age_cycle']==2

    

    taz_2020_catalog.loc[
        (age_cycle_time_condition) &
        (age_cycle_condition) &
        (sector_condition) &
        (taz_2020_catalog['changed_this_cycle']==0) &
        (taz_with_aprt_diversity_condition),['age_cycle','explanation_{}'.format(y1),'changed_this_cycle']
    ]=[3,'| natural aging to a diverse neighborhood |',1]
    
    taz_2020_catalog.loc[
        (age_cycle_time_condition) &
        (age_cycle_condition) &
        (sector_condition) &
        (taz_with_aprt_diversity_condition),'age_cycle_time'
    ]=taz_2020_catalog['age_cycle_time']+5

    sector_condition=taz_2020_catalog['sector_for_age']=='Jewish'

    age_cycle_condition = taz_2020_catalog['age_cycle']==4

    

    taz_2020_catalog.loc[
        (age_cycle_time_condition) &
        (age_cycle_condition) &
        (sector_condition) &
        (taz_2020_catalog['changed_this_cycle']==0) &
        (taz_with_aprt_diversity_condition),['age_cycle','explanation_{}'.format(y1),'changed_this_cycle']
    ]=[3,'| old to a diverse neighborhood |',1]
    
    taz_2020_catalog.loc[
        (age_cycle_time_condition) &
        (age_cycle_condition) &
        (sector_condition) &
        (taz_with_aprt_diversity_condition),'age_cycle_time'
    ]=taz_2020_catalog['age_cycle_time']+5
   
    
    sector_condition=taz_2020_catalog['sector_for_age']=='U_Orthodox'

    age_cycle_condition = taz_2020_catalog['age_cycle']==1


    taz_2020_catalog.loc[
        (age_cycle_time_condition) &
        (sector_condition) &
        (taz_2020_catalog['changed_this_cycle']==0) &
        (age_cycle_condition),['age_cycle','explanation_{}'.format(y1),'changed_this_cycle']
    ]=[2 ,'| natural aging |',1]
    
    
    taz_2020_catalog.loc[
        (age_cycle_time_condition) &
        (sector_condition) &
        (age_cycle_condition),'age_cycle_time'
    ]=taz_2020_catalog['age_cycle_time']+5

    age_cycle_condition = taz_2020_catalog['age_cycle']==2

    taz_2020_catalog.loc[
        (age_cycle_time_condition) &
        (taz_2020_catalog['changed_this_cycle']==0) &
        (age_cycle_condition) &
        (sector_condition) &
        (taz_with_aprt_diversity_condition),['age_cycle','explanation_{}'.format(y1),'changed_this_cycle']
    ]=[3,'| natural aging to a diverse neighborhood |',1]
    
    
    taz_2020_catalog.loc[
        (age_cycle_time_condition) &
        (age_cycle_condition) &
        (sector_condition) &
        (taz_with_aprt_diversity_condition),'age_cycle_time'
    ]=taz_2020_catalog['age_cycle_time']+5
    
    
    taz_2020_catalog.loc[
        (age_cycle_time_condition) &
         (taz_2020_catalog['changed_this_cycle']==0) &
        (age_cycle_condition) &
        (sector_condition) &
        ~(taz_with_aprt_diversity_condition),['age_cycle','explanation_{}'.format(y1),'changed_this_cycle']
    ]=[4,'| natural aging |',1]
    
    age_cycle_condition = taz_2020_catalog['age_cycle']==4
    
    taz_2020_catalog.loc[
        (age_cycle_time_condition) &
        (taz_2020_catalog['changed_this_cycle']==0) &
        (age_cycle_condition) &
        (sector_condition) &
        (taz_with_aprt_diversity_condition),['age_cycle','explanation_{}'.format(y1),'changed_this_cycle']
    ]=[3,'| old to a diverse neighborhood |',1]
    
    
    taz_2020_catalog.loc[
        (age_cycle_time_condition) &
        (age_cycle_condition) &
        (sector_condition) &
        (taz_with_aprt_diversity_condition),'age_cycle_time'
    ]=taz_2020_catalog['age_cycle_time']+5
    
    
   

    taz_2020_catalog.loc[
        (age_cycle_time_condition) &
        (age_cycle_condition) &
        (sector_condition) &
        ~(taz_with_aprt_diversity_condition),'age_cycle_time'
    ]=taz_2020_catalog['age_cycle_time']+5

    taz_2020_catalog.loc[(taz_2020_catalog['changed_this_cycle']==0),'age_cycle_time'
    ]=taz_2020_catalog['age_cycle_time']+5
    
    taz_2020_catalog['changed_this_cycle']=0

    
    taz_2020_catalog['current_aprt']=taz_2020_catalog['current_aprt']+taz_2020_catalog['{}'.format(y)]
    taz_2020_catalog['current_aprt_{}'.format(y1)]= taz_2020_catalog['current_aprt']
    taz_2020_catalog['age_cycle_{}'.format(y1)]= taz_2020_catalog['age_cycle']
    taz_2020_catalog['age_cycle_time_{}'.format(y1)]= taz_2020_catalog['age_cycle_time']
    taz_2020_catalog['precent_add_to_current_{}'.format(y1)]= taz_2020_catalog['precent_add_to_current']

C:\Users\dpere\AppData\Local\Temp\ipykernel_25304\617637879.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value ' | new diverse neighborhood | ' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  taz_2020_catalog.loc[
C:\Users\dpere\AppData\Local\Temp\ipykernel_25304\617637879.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value ' | new diverse neighborhood | ' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  taz_2020_catalog.loc[
C:\Users\dpere\AppData\Local\Temp\ipykernel_25304\617637879.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value ' | new diverse neighborhood | ' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  taz_2020_catalog.loc[
C:\Users\d

In [20]:
file_date=pd.Timestamp.today().strftime('%y%m%d')

## ייצוא

In [21]:
taz_2020_catalog['hh_size_2020']=taz_2020_catalog['pop_without_dorms_yeshiva']/taz_2020_catalog['aprt_20']

In [22]:
for y1,sy,y_b, in zip(only_years, singal_year,y_before):
    taz_2020_catalog.loc[(taz_2020_catalog['age_cycle_{}'.format(y1)]==taz_2020_catalog['age_cycle_{}'.format(y_b)])&(taz_2020_catalog['age_cycle_{}'.format(y1)]==3),'hh_size_{}'.format(y1)]=taz_2020_catalog['hh_size_{}'.format(y_b)]
    taz_2020_catalog.loc[(taz_2020_catalog['age_cycle_{}'.format(y1)]==taz_2020_catalog['age_cycle_{}'.format(y_b)])&(taz_2020_catalog['age_cycle_{}'.format(y1)]==3),'hh_size_{}'.format(y1)]=taz_2020_catalog['hh_size_{}'.format(y1)]+0.15
    

In [23]:
col_age=[ 'pop_0',
 'pop_10',
 'pop_15',
 'pop_20',
 'pop_25',
 'pop_30',
 'pop_35',
 'pop_40',
 'pop_45',
 'pop_5',
 'pop_50',
 'pop_55',
 'pop_60',
 'pop_65',
 'pop_70',
 'pop_75up']

In [24]:
for y1,sy,y_b, in zip(only_years, singal_year,y_before):
    
    col_taz=['Taz_num','current_aprt_{}'.format(y1),'explanation_{}'.format(y1),'age_cycle_{}'.format(y1),'hh_size_{}'.format(y1),'sector_for_age']
    
    left_col=['sector_for_age','age_cycle_{}'.format(y1)]
    right_col=['sector','age_cycle']

    df=taz_2020_catalog.reset_index()[col_taz].merge(age_catalog,left_on=left_col,right_on=right_col,how='left').set_index('Taz_num')
    

    df['pop']=df['current_aprt_{}'.format(y1)]*df['hh_size_{}'.format(y1)]
    df.loc[df['hh_size_{}'.format(y1)].isna(),'pop']=df['current_aprt_{}'.format(y1)]*df['hh_size']
    
        
    df['pop_{}'.format(sy)]=df['pop']
    
    df['aprt_{}'.format(sy)]=df['current_aprt_{}'.format(y1)]
    
    df[col_age]=df[col_age].multiply(df['pop'], axis="index")    
    
    file_save=r'{}\Intermediates'.format(software_data_folder_location)
    save_excel_path=r'{}\{}_forecast_pop_jewish_{}_jtmt.xlsx'.format(file_save,file_date,y1)
    
    df.to_excel(save_excel_path,index=True)
    
    taz_2020_catalog=taz_2020_catalog.join(df[['pop_{}'.format(sy),'aprt_{}'.format(sy)]])
    
 

In [25]:
taz_2020_catalog['add_aprt_till_2050']=taz_2020_catalog[['add_aprt_2020_2025',
 'add_aprt_2025_2030',
 'add_aprt_2030_2035',
 'add_aprt_2035_2040',
 'add_aprt_2040_2045',
 'add_aprt_2045_2050']].sum(axis=1)

In [26]:
col_order=['Taz_name','Muni_Heb','zonetype',
 'taz_with_aprt_diversity',
 'taz_expensive',
 'sector_for_age',
 'explanation_2020_2025',
 'explanation_2025_2030',
 'explanation_2030_2035',
 'explanation_2035_2040',
 'explanation_2040_2045',
 'explanation_2045_2050',
 'aprt_20','aprt_2025','aprt_2030','aprt_2035','aprt_2040','aprt_2045'
,'aprt_2050', 'add_aprt_2020_2025',
 'add_aprt_2025_2030',
 'add_aprt_2030_2035',
 'add_aprt_2035_2040',
 'add_aprt_2040_2045',
 'add_aprt_2045_2050',
 'add_aprt_till_2050',
 'pop_without_dorms_yeshiva',
 'pop_2025',
 'pop_2030',
 'pop_2035',
 'pop_2040',
 'pop_2045',
 'pop_2050',
 'classification_name',
 'age_cycle_2020',
 'age_cycle_2020_2025',
 'age_cycle_2025_2030',
 'age_cycle_2030_2035',
 'age_cycle_2035_2040',
 'age_cycle_2040_2045',
 'age_cycle_2045_2050',
 'age_cycle_time_2020_2025',
 'age_cycle_time_2025_2030',
 'age_cycle_time_2030_2035',
 'age_cycle_time_2035_2040',
 'age_cycle_time_2040_2045',
 'age_cycle_time_2045_2050',
 'precent_add_to_current_2020_2025',
 'precent_add_to_current_2025_2030',
 'precent_add_to_current_2030_2035',
 'precent_add_to_current_2035_2040',
 'precent_add_to_current_2040_2045',
 'precent_add_to_current_2045_2050']

In [27]:
file_save=r'{}\Intermediates'.format(software_data_folder_location)
save_excel_path=r'{}\{}_forecast_pop_jewish_till_2050_jtmt.xlsx'.format(file_save,file_date)

In [28]:
taz_2020_catalog[col_order].to_excel(save_excel_path)

In [29]:
taz_2020_catalog_pivot=taz_2020_catalog.pivot_table(index='Muni_Heb',aggfunc=sum)

C:\Users\dpere\AppData\Local\Temp\ipykernel_25304\3239392360.py:1: FutureWarning: The provided callable <built-in function sum> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  taz_2020_catalog_pivot=taz_2020_catalog.pivot_table(index='Muni_Heb',aggfunc=sum)


In [30]:
taz_2020_catalog_pivot['add_pop_till_2050']=taz_2020_catalog_pivot['pop_2050']-taz_2020_catalog_pivot['pop_without_dorms_yeshiva']

In [31]:
taz_2020_catalog_pivot['precent_growth_till_2050']=round(taz_2020_catalog_pivot['pop_2050']/taz_2020_catalog_pivot['pop_without_dorms_yeshiva'],2)

In [32]:
col_order=['pop_without_dorms_yeshiva',
 'pop_2025',
 'pop_2030',
 'pop_2035',
 'pop_2040',
 'pop_2045',
 'pop_2050','add_pop_till_2050','precent_growth_till_2050','aprt_20','aprt_2025','aprt_2030','aprt_2035','aprt_2040','aprt_2045'
,'aprt_2050',
 'add_aprt_2020_2025',
 'add_aprt_2025_2030',
 'add_aprt_2030_2035',
 'add_aprt_2035_2040',
 'add_aprt_2040_2045',
 'add_aprt_2045_2050',
 'add_aprt_till_2050' ]

In [33]:
save_excel_path=r'{}\{}_pivot_by_muni_forecast_pop_jewish_till_2050_jtmt.xlsx'.format(file_save,file_date)

In [34]:
taz_2020_catalog_pivot[col_order].query('index!="0"').to_excel(save_excel_path)